In [3]:
###Q3 final
sc.stop()
from pyspark import SparkContext
sc = SparkContext(appName="exercise")

# read csv file
temperature_file=sc.textFile("temperature-readings-tiny.csv")
lines = temperature_file.map(lambda line:line.split(";"))

# map necessary keys and values
# (year-month, temperature), stationNumber
year_temperature = lines.map(lambda x:(x[1][0:7],float(x[3]),x[0]))
year_temperature = year_temperature.filter(lambda x: int(x[0][0:4])>=1960 and int(x[0][0:4])<=2014)

# (year-month, stationNumber), #readings
countSource = year_temperature.map(lambda x:((x[0],x[2]),1))
counts=countSource.reduceByKey(lambda x,y: x+y)

# (year-month, stationNumber), tempSum
temperature = year_temperature.map(lambda x:((x[0],x[2]),x[1]))
tempSum=temperature.reduceByKey(lambda v1,v2:v1+v2)

# (year, month, stationNumber, TempAve)
aveSource = tempSum.join(counts)
aveTemp = aveSource.map(lambda x:(x[0][0][0:4],x[0][0][5:7],x[0][1],round(x[1][0]/x[1][1],2)))
aveTemp = aveTemp.sortBy(lambda x: x[3], ascending = False)

# print(*aveTemp.take(20),sep="\n")
averagetemp.saveAsTextFile("aveTemp")

('2014', '05', '102150', 50.0)
('2014', '07', '102160', 34.5)
('2014', '05', '102130', 32.0)
('2014', '07', '102170', 19.66)
('1960', '06', '102190', 15.66)
('2014', '06', '102170', 14.44)
('1960', '07', '102190', 14.21)
('2014', '08', '102170', 13.87)
('1960', '08', '102190', 13.41)
('2014', '05', '102160', 11.5)
('1960', '05', '102190', 10.77)
('2014', '05', '102170', 10.76)
('1961', '05', '102190', 10.59)
('1960', '09', '102190', 9.37)
('2014', '09', '102170', 8.58)
('2014', '10', '102170', 7.11)
('2014', '04', '102170', 4.78)
('1961', '04', '102190', 4.51)
('1961', '03', '102190', 2.71)
('2014', '11', '102170', 2.53)
